# Bus Trip Table Scenario

## Lets import bus trip data

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

## Lets import bus, location and trip data

In [2]:
bus = pd.read_excel('../data/Bus-Trip-Data.xlsx',sheet_name='bus')
bus

,bus_id,bus_code,bus_registration_no,bus_type,registered_on,no_of_seats
0,150,BLR001,KA-51-A-1234,Sleeper,2021-08-15,35
1,151,BLR002,KA-51-A-1235,AC,2023-04-16,35
2,152,BLR003,KA-51-A-1236,Sleeper,2023-11-16,40
3,153,BLR004,KA-51-A-1237,AC,2024-04-16,36
4,154,BLR005,KA-51-A-1238,Sleeper,2024-04-30,35
5,155,BLR000,KA-51-A-1200,Sleeper,2019-09-15,36
6,156,BLR007,KA-51-A-1201,Normal,2024-04-16,36


In [3]:
location = pd.read_excel('../data/Bus-Trip-Data.xlsx',sheet_name='location')
location

,loc_id,loc_code,loc_city,loc_state,loc_zip_code
0,10,BLR,Bangalore,Karanataka,560076
1,11,CHN,Chennai,Tamil Nadu,600001
2,9,MYS,Mysore,Karanataka,570001
3,8,MAN,Mangalore,Karanataka,575004
4,7,HAS,Hassan,Karanataka,573201
5,6,MUB,Mumbai,Maharashtra,400001
6,5,KOL,Kolar,Karanataka,563101


In [4]:
trip = pd.read_excel('../data/Bus-Trip-Data.xlsx',sheet_name='trip')
trip

,trip_id,trip_date,bus_id,from_loc_id,to_loc_id
0,75,2022-07-15,150,10,11
1,76,2025-05-16,151,10,8
2,77,2025-05-18,152,10,7
3,78,2025-05-20,151,8,6
4,79,2025-05-26,153,9,11
5,80,2025-05-18,154,11,10


### Question 1: Write a query to list the states and city count. Consider only states which has more than 4 cities

In [5]:
# Lets use groupby to group according to the states and select states which has rows more than 4
state_city_count = location.pivot_table(values='loc_city',index='loc_state',aggfunc=np.size).reset_index()
state_city_count

,loc_state,loc_city
0,Karanataka,5
1,Maharashtra,1
2,Tamil Nadu,1


In [6]:
# count of states which has location more than 5 cities 
state_city_count[state_city_count['loc_city']>4]

,loc_state,loc_city
0,Karanataka,5


We have karnataka as a state which has city count more than 5

### Question 2: Display the bus code and registration_no for all the bus_type 'AC' or 'Sleeper' and registered in last year April month

In [7]:
bus_data=bus.copy()

In [8]:
bus_month_bool=(bus_data['registered_on'].dt.year==2024) & (bus_data['registered_on'].dt.month==4)
bus_type_bool = (bus_data['bus_type']=='AC')|(bus_data['bus_type']=='Sleeper')

In [9]:
bus[bus_month_bool & bus_type_bool]

,bus_id,bus_code,bus_registration_no,bus_type,registered_on,no_of_seats
3,153,BLR004,KA-51-A-1237,AC,2024-04-16,36
4,154,BLR005,KA-51-A-1238,Sleeper,2024-04-30,35


### Question3: Display the number of buses registered in each year (consider only 5 years including current year)

In [10]:
# Bus which has become more than 5 years 
bus_data[pd.Timestamp.today().year-bus_data['registered_on'].dt.year>=5]

,bus_id,bus_code,bus_registration_no,bus_type,registered_on,no_of_seats
5,155,BLR000,KA-51-A-1200,Sleeper,2019-09-15,36


In [11]:
# Bus registered in each each not older than 5 years.
bus_data['registered_year']=bus_data['registered_on'].dt.year
bus_registered_year_wise= bus_data.pivot_table(index='registered_year',values='bus_registration_no',aggfunc=np.size).reset_index()
bus_registered_year_wise_5 = bus_registered_year_wise[pd.Timestamp.today().year-bus_registered_year_wise['registered_year']<=5]
bus_registered_year_wise_5=bus_registered_year_wise_5.rename({'bus_registration_no':'bus_registered_count'},axis=1)
bus_registered_year_wise_5.sort_values(by='bus_registered_count',ascending=False).reset_index(drop=True)

,registered_year,bus_registered_count
0,2024,3
1,2023,2
2,2021,1


### Question4: Find all the trips we have in the current week which are starting from Bangalore

In [12]:
# Creating boolean to select trip details of this week using  ISO years always start with Monday and ends on sunday.

trip_this_week_bool = (trip['trip_date'].dt.isocalendar().week - pd.Timestamp.today().date().isocalendar().week )==0
trip_bangalore_bool=trip['from_loc_id']==10

trip[trip_this_week_bool & trip_bangalore_bool]

,trip_id,trip_date,bus_id,from_loc_id,to_loc_id


### Question5: Find all the trips where the destination is Mumbai 

In [13]:
# Mumbai location id = 6 according to location table

trip[trip['to_loc_id']==6].reset_index(drop=True)

,trip_id,trip_date,bus_id,from_loc_id,to_loc_id
0,78,2025-05-20,151,8,6


### Question6: Display the trip id and trip_date which are going from Mysore to Chennai

In [14]:
# from location id is mysore having location id = 9 and to location chennai having location id = 11
trip[(trip['from_loc_id']==9) & (trip['to_loc_id']==11)].reset_index(drop=True)

,trip_id,trip_date,bus_id,from_loc_id,to_loc_id
0,79,2025-05-26,153,9,11


### Question6: Display trip_id, trip_date, bus_registration_no, trip_starting_city, trip_ending_city, trip_starting_state, trip_ending_state

In [15]:
trip_bus_merge = pd.merge(left=trip,right=bus,on=['bus_id'],how='inner')
trip_bus_fromloc_merge = pd.merge(left=trip_bus_merge,right=location,left_on='from_loc_id',right_on='loc_id',how='inner')

In [16]:
trip_bus_fromtoloc_merge = pd.merge(left=trip_bus_fromloc_merge,right=location,left_on='to_loc_id',
                                    right_on='loc_id',how='left',suffixes=("_start","_end"))
trip_bus_fromtoloc_merge

,trip_id,trip_date,bus_id,from_loc_id,to_loc_id,bus_code,bus_registration_no,bus_type,registered_on,no_of_seats,loc_id_start,loc_code_start,loc_city_start,loc_state_start,loc_zip_code_start,loc_id_end,loc_code_end,loc_city_end,loc_state_end,loc_zip_code_end
0,75,2022-07-15,150,10,11,BLR001,KA-51-A-1234,Sleeper,2021-08-15,35,10,BLR,Bangalore,Karanataka,560076,11,CHN,Chennai,Tamil Nadu,600001
1,76,2025-05-16,151,10,8,BLR002,KA-51-A-1235,AC,2023-04-16,35,10,BLR,Bangalore,Karanataka,560076,8,MAN,Mangalore,Karanataka,575004
2,77,2025-05-18,152,10,7,BLR003,KA-51-A-1236,Sleeper,2023-11-16,40,10,BLR,Bangalore,Karanataka,560076,7,HAS,Hassan,Karanataka,573201
3,78,2025-05-20,151,8,6,BLR002,KA-51-A-1235,AC,2023-04-16,35,8,MAN,Mangalore,Karanataka,575004,6,MUB,Mumbai,Maharashtra,400001
4,79,2025-05-26,153,9,11,BLR004,KA-51-A-1237,AC,2024-04-16,36,9,MYS,Mysore,Karanataka,570001,11,CHN,Chennai,Tamil Nadu,600001
5,80,2025-05-18,154,11,10,BLR005,KA-51-A-1238,Sleeper,2024-04-30,35,11,CHN,Chennai,Tamil Nadu,600001,10,BLR,Bangalore,Karanataka,560076


In [17]:
trip_details_final = trip_bus_fromtoloc_merge[['trip_id','trip_date','bus_registration_no',
                                               'loc_city_start','loc_city_end','loc_state_start','loc_state_end']]

In [18]:
trip_details_final=trip_details_final.rename({'loc_city_start':'trip_starting_city','loc_city_end':'trip_ending_city',
                                              'loc_state_start':'trip_starting_state','loc_state_end':'trip_ending_state'},axis=1)

In [19]:
trip_details_final

,trip_id,trip_date,bus_registration_no,trip_starting_city,trip_ending_city,trip_starting_state,trip_ending_state
0,75,2022-07-15,KA-51-A-1234,Bangalore,Chennai,Karanataka,Tamil Nadu
1,76,2025-05-16,KA-51-A-1235,Bangalore,Mangalore,Karanataka,Karanataka
2,77,2025-05-18,KA-51-A-1236,Bangalore,Hassan,Karanataka,Karanataka
3,78,2025-05-20,KA-51-A-1235,Mangalore,Mumbai,Karanataka,Maharashtra
4,79,2025-05-26,KA-51-A-1237,Mysore,Chennai,Karanataka,Tamil Nadu
5,80,2025-05-18,KA-51-A-1238,Chennai,Bangalore,Tamil Nadu,Karanataka


### Question7:Find the buses which are serving interstate (between states -- example, trips from KA to AP) trips in the current month

In [20]:
current_month = dt.datetime.now().month
trip_details_final[(trip_details_final['trip_date'].dt.month==current_month)&
    (trip_details_final['trip_starting_state']!=trip_details_final['trip_ending_state'])].reset_index(drop=True)

,trip_id,trip_date,bus_registration_no,trip_starting_city,trip_ending_city,trip_starting_state,trip_ending_state
